# Driving a Financial Valuation from a Design Scenario

```{warning}
The following is a work in progress. More content will be added as produced.
```

As seen in
[Loading a Design Scenario](../load_design_scenario.ipynb),
a design scenario's object model (a
[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph) including all
its spatial and semantic relationships) can be loaded via Speckle from any of
Speckle's supported [connectors](https://speckle.guide/user/connectors.html), so
long as those objects (`Entity`s and `Relationship`s) have been assigned.

In this notebook, we will demonstrate how to use the object model (graph) to
drive a financial valuation of the design scenario. This is done by calculating
cash flows per `Entity`, and then using the `Relationship`s to aggregate those
`Flow`s into appropriate `Stream`s.

## Load the Design Scenario
We will use the same design scenario as in the previous notebook.

In [1]:
import os

import networkx as nx
import locale
import pandas as pd
from IPython.display import IFrame
import plotly.io as pio
import plotly.express as px
import plotly.offline as py
import pprint as pp
import rangekeeper as rk

In [2]:
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN'))
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)


 SpeckleClient( server: https://speckle.xyz, authenticated: True )


### Inspect the Model Graph

In [3]:
model = speckle.get_commit(stream_id=stream_id)
parsed = rk.api.Speckle.parse(base=model['@scenario'])
scenario = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='design_scenario',
    type='scenario')

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


In [4]:
scenario.plot(name='assets/design_scenario')
IFrame(src="./design_scenario.html", width='100%', height=800)

assets/design_scenario.html


## Assigning and Aggregating `Flow`s per `Entity`
Now we can organise the two main categories of cash flows (costs and revenues)
according to the `Relationship` types in the design scenario.

In this example, revenue-based cash flows are generated by `Entity`s that have a
 "sellable" (or "leasable") area -- an NSA or NLA measure.

We can query the graph to find them:

### Revenue-producing Entities
Revenue-producing `Entity`s will be aggregated by their spatial containment. So
we can first slice our graph to a tree of nodes that have "contains"
relationships:

In [5]:
spatial_containment = rk.graph.Assembly.from_graph(
    graph=scenario.graph.edge_subgraph(
        [edge for edge in scenario.graph.edges(keys=True) if edge[2] == 'spatiallyContains']),
    name='spatial_containment',
    type='subgraph')

Note: we anticipate that the spatial decomposition is completely hierarchical
(i.e., no spaces overlap, which means no spaces have multiple parents). We can
check this by testing whether the containment graph is a "tree" (or
"arborescence": see [NetworkX Tree](https://networkx.org/documentation/stable/reference/algorithms/tree.html#tree)).

In [6]:
nx.is_arborescence(spatial_containment.graph)

True

In [7]:
spatial_containment.plot(name='assets/spatial_containment')
IFrame(src="./spatial_containment.html", width='100%', height=800)

assets/spatial_containment.html


#### Simple Aggregation
Now that we have verified this, we can continue with a simple aggregation. We
to aggregate up through our hierarchy all `Entity`s that have a Gross Floor Area
('gfa') property:

In [8]:
spatial_containment.aggregate(
    property='gfa',
    label='subtotal_gfa')

We can tabulate this by converting the graph to a pandas `DataFrame`:

In [ ]:
spatial_containment.to_DataFrame()

,entityId,subtotal_gfa,name,id,type,parent,children,use,ffl,gfa,number
003c1ee8-4450-4fb1-98bd-54d11408d99e,003c1ee8-4450-4fb1-98bd-54d11408d99e,45816.419959,development,06f4b837038f2cc3edf4244fceb946f3,development,None,"[eb42cd14-73ab-41fc-99c0-c46631ee1208, dac040a...",NaN,NaN,NaN,NaN
eb42cd14-73ab-41fc-99c0-c46631ee1208,eb42cd14-73ab-41fc-99c0-c46631ee1208,19605.790919,buildingA,a648fcc8f96e4160b0c35953139e4211,building,003c1ee8-4450-4fb1-98bd-54d11408d99e,"[655b2dfe-5c18-4d68-95f0-f5d27804430d, 47dfa8b...",NaN,NaN,NaN,NaN
dac040a5-2605-4cd0-a2fb-40ebb85450d9,dac040a5-2605-4cd0-a2fb-40ebb85450d9,12773.744211,plinth,8cc56cf9901af95f74b642f7a1950e6f,building,003c1ee8-4450-4fb1-98bd-54d11408d99e,"[c7dfbeaf-1b8c-424c-bad0-0b2b5975af6e, 1993c55...",NaN,NaN,NaN,NaN
8c7c9be7-fd8f-4527-ae90-24e15c3581db,8c7c9be7-fd8f-4527-ae90-24e15c3581db,13436.884829,buildingB,bfc0db6a88d730984d4eead94ac5a2e6,building,003c1ee8-4450-4fb1-98bd-54d11408d99e,"[23b3acb0-8acb-4254-b54c-e76b4b7b8fe9, 1235045...",NaN,NaN,NaN,NaN
f7e2fa84-54a6-4fd4-ae93-f339947035b6,f7e2fa84-54a6-4fd4-ae93-f339947035b6,0.000000,utilities,aabdd3fcac0dd1e5c793235001dd945b,utilities,003c1ee8-4450-4fb1-98bd-54d11408d99e,[],NaN,NaN,NaN,NaN
655b2dfe-5c18-4d68-95f0-f5d27804430d,655b2dfe-5c18-4d68-95f0-f5d27804430d,14691.140177,buildingAoffice,f3fcf8533f26a84803e72de91ec51d47,space,eb42cd14-73ab-41fc-99c0-c46631ee1208,"[6bfd12d7-72d7-417f-bc31-1bef4ffaadb0, f25ef6d...",office,NaN,NaN,NaN
47dfa8b6-e213-4099-9f5b-834df0ca34b0,47dfa8b6-e213-4099-9f5b-834df0ca34b0,1123.715326,buildingAretail,6e4bdf36f792bd089039c6ef4c5cf23e,space,eb42cd14-73ab-41fc-99c0-c46631ee1208,[037235ef-f5ef-4763-b652-13d59dc03a34],retail,NaN,NaN,NaN
da796c3e-94b0-4175-8c6f-ac63e5e60714,da796c3e-94b0-4175-8c6f-ac63e5e60714,2296.143091,buildingAresidential,5e079a3b71d254fc2eba55f62a1188d1,space,eb42cd14-73ab-41fc-99c0-c46631ee1208,"[a987a169-2605-4308-9462-8d22eb6df2be, d2db8f8...",residential,NaN,NaN,NaN
08e12c72-4450-424f-8ff4-a4da7246ee05,08e12c72-4450-424f-8ff4-a4da7246ee05,534.530238,buildingAretail,ff0fdddd8d8669831c2c1f6a0bad6640,space,eb42cd14-73ab-41fc-99c0-c46631ee1208,[b5466f16-c071-41a5-93b1-0d794930e62d],retail,NaN,NaN,NaN
634c5fb4-f204-4287-8c80-f23371056f1c,634c5fb4-f204-4287-8c80-f23371056f1c,960.262087,buildingAparking,7de6c744113bcb82c54b2c10d12a451d,space,eb42cd14-73ab-41fc-99c0-c46631ee1208,[02b101a5-0ad5-4f8d-9f0b-f65787d4073f],parking,NaN,NaN,NaN


We can also plot this as a hierarchical pie chart (a.k.a. 'suburst' chart):

In [10]:
fig = spatial_containment.sunburst(property='subtotal_gfa')
filename='spatial_containment_sunburst.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800)

#### Aggregating a (Financial) Calculation
A more complex aggregation involves collecting specific cash `Flow`s into
`Stream`s per `Entity`/`Assembly`; for instance, we may want to aggregate all
revenues generated by lettable floorspace into their parent `Entity`s, in order
to analyse or compare the performance of the scenario at different resolutions.

We start by identifying which `Entity`s are 'floor' types and have an area
measurement:

In [11]:
floors = [entity for (entityId, entity) in spatial_containment.get_entities().items() if entity['type'] == 'floor' and hasattr(entity, 'gfa')]

For each floor we can project a set of simple cash flows, in a similar manner
to [A Basic DCF Valuation](../basic_dcf.ipynb):

First initialize our locale, and a set of parameters, including some around
area efficiency ratios, initial incomes per area, and growth rates:

In [12]:
locale.setlocale(locale.LC_ALL, 'en_au')
units = rk.measure.Index.registry
currency = rk.measure.register_currency(registry=units)
period_type = rk.periodicity.Type.YEAR

In [13]:
efficiency_ratios = {
    'office': 0.88,
    'retail': 0.75,
    'residential': 0.82,
    'parking': 0.93,
    }
initial_income_per_area_pa = {
    'office': 750,
    'retail': 1000,
    'residential': 600,
    'parking': 0,
    }
growth_rates = {
    'office': 0.02,
    'retail': 0.05,
    'residential': 0.035,
    'parking': 0,
    }
vacancy_rates = {
    'office': 0.075,
    'retail': 0.5,
    'residential': 0.02,
    'parking': 0,
    }

In [14]:
params = {
    'start_date': pd.Timestamp('2001-01-01'),
    'num_periods': 10,
    'period_type': rk.periodicity.Type.YEAR,
    }

##### `Stream`s of Revenue `Flow`s per `Entity`:
It is now simple to construct a set of cash flows per `Entity` in our design
scenario. We will use a similar structure to the previous notebooks:

First, we will define a `Revenues` class that holds the parameters:

In [15]:
class Revenues:
    def __init__(self, params: dict):
        self.params = params

Next we add our `Span` information:

In [16]:
@rk.update_class(Revenues)
class Revenues:
    def init_spans(self):
        self.calc_span = rk.span.Span.from_num_periods(
            name='Span to Calculate Reversion',
            date=self.params['start_date'],
            period_type=self.params['period_type'],
            num_periods=self.params['num_periods'] + 1)
        self.acq_span = rk.span.Span.from_num_periods(
            name='Acquisition Span',
            date=rk.periodicity.offset_date(
                self.params['start_date'],
                num_periods=-1,
                period_type=self.params['period_type']),
            period_type=self.params['period_type'],
            num_periods=1)
        self.span = self.calc_span.shift(
            name='Span',
            num_periods=-1,
            period_type=self.params['period_type'],
            bound='end')

And then we add Revenue `Flow`s:

In [17]:
@rk.update_class(Revenues)
class Revenues:
    def init_flows(self):
        self.pgi = rk.flux.Flow.from_projection(
            name='Potential Gross Income',
            value=self.params['initial_income'],
            proj=rk.projection.Extrapolation(
            form=rk.extrapolation.Compounding(
                rate=self.params['growth_rate']),
            sequence=self.calc_span.to_index(period_type=self.params['period_type'])),
            units=currency.units)
        self.vacancy = rk.flux.Flow(
            name='Vacancy Allowance',
            movements=self.pgi.movements * -self.params['vacancy_rate'],
            units=currency.units)
        self.egi = rk.flux.Stream(
            name='Effective Gross Income',
            flows=[self.pgi, self.vacancy],
            period_type=self.params['period_type'])

Now we need to add `Entity`-specific parameters to the parameters dictionary,
and run  the `init_spans()` and `init_flows()` methods per floor `Entity`:

(Note we place the resultant `Stream` in the `events` attribute of the `Entity`)

In [18]:
for floor in floors:
    parent = floor.get_relatives(
        relationship_type='spatiallyContains',
        outgoing=False,
        assembly=spatial_containment)[0]
    floor['use'] = parent['use']

    floor.params = params.copy()
    floor.params['initial_income'] = initial_income_per_area_pa[floor['use']] * floor['gfa']
    floor.params['growth_rate'] = growth_rates[floor['use']]
    floor.params['vacancy_rate'] = vacancy_rates[floor['use']]

    floor.events = {}

    revenues = Revenues(floor.params)
    revenues.init_spans()
    revenues.init_flows()
    floor.events = {
        'egi': revenues.egi,
        }

##### Aggregating by Container
Since we have the "containment" hierarchy, we can aggregate the `Revenues` per
containing `Assembly`:

*Rangekeeper* provides some helper methods to assist in the aggregation of
`Stream`s and `Flow`s; specifically, since the `Entity.aggregate()` method
updates a specified `Entity`'s property with a dictionary of subentity
properties, keyed by their respective `entityId`s, we can use the
`aggregate_events()` function to support the summation of `Stream`s and `Flow`s
into a single `Stream` per key in the `Entity`'s `events` property:

In [19]:
def aggregate_egis(**kwargs):
    return rk.graph.Entity.aggregate_events(
        **kwargs,
        key='egi',
        period_type=period_type)

This function is passed into the `Entity.aggregate()` method as the `function`
parameter:

In [20]:
aggregation = spatial_containment.aggregate(
    function=aggregate_egis,
    property='events',
    label='aggregate_egi',
    relationship_type='spatiallyContains')

In [21]:
floors[1]['events']['egi']

date,Potential Gross Income,Vacancy Allowance
2001,"$1,086,691.51","-$81,501.86"
2002,"$1,108,425.34","-$83,131.90"
2003,"$1,130,593.84","-$84,794.54"
2004,"$1,153,205.72","-$86,490.43"
2005,"$1,176,269.83","-$88,220.24"
2006,"$1,199,795.23","-$89,984.64"
2007,"$1,223,791.14","-$91,784.34"
2008,"$1,248,266.96","-$93,620.02"
2009,"$1,273,232.30","-$95,492.42"
2010,"$1,298,696.94","-$97,402.27"


We can now inspect the `aggregate_egi` property of an `Entity`. Let's use the
office compartment in Building A:

In [22]:
buildingAoffice = [e for (id, e) in spatial_containment.get_entities().items() if e.name == 'buildingAoffice'][0]
print(buildingAoffice.name)
buildingAoffice['aggregate_egi']

buildingAoffice


date,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>,Effective Gross Income for <built-in function id>
2001,"$1,145,271.71","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64","$1,005,189.64"
2002,"$1,168,177.14","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44","$1,025,293.44"
2003,"$1,191,540.69","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30","$1,045,799.30"
2004,"$1,215,371.50","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29","$1,066,715.29"
2005,"$1,239,678.93","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60","$1,088,049.60"
2006,"$1,264,472.51","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59","$1,109,810.59"
2007,"$1,289,761.96","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80","$1,132,006.80"
2008,"$1,315,557.20","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94","$1,154,646.94"
2009,"$1,341,868.34","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88","$1,177,739.88"
2010,"$1,368,705.71","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67","$1,201,294.67"


We can also easily sum and collapse that aggregation, as it is a Stream`:

In [23]:
buildingAoffice['aggregate_egi'].sum()

date,egi for 655b2dfe-5c18-4d68-95f0-f5d27804430d Aggregation (sum)
2001-12-31 00:00:00,"$10,191,978.50"
2002-12-31 00:00:00,"$10,395,818.07"
2003-12-31 00:00:00,"$10,603,734.43"
2004-12-31 00:00:00,"$10,815,809.12"
2005-12-31 00:00:00,"$11,032,125.30"
2006-12-31 00:00:00,"$11,252,767.81"
2007-12-31 00:00:00,"$11,477,823.16"
2008-12-31 00:00:00,"$11,707,379.63"
2009-12-31 00:00:00,"$11,941,527.22"
2010-12-31 00:00:00,"$12,180,357.76"


In [24]:
buildingAoffice['aggregate_egi'].sum().collapse()

date,egi for 655b2dfe-5c18-4d68-95f0-f5d27804430d Aggregation (sum)
2011-12-31 00:00:00,"$124,023,285.90"


In [25]:
for (entityId, entity) in spatial_containment.get_entities().items():
    entity['aggregate_egi_value'] = entity['aggregate_egi'].sum().collapse().movements.to_numpy()[0] if entity['aggregate_egi'] is not None else None

In [26]:
fig = spatial_containment.sunburst(property='aggregate_egi_value')
filename='aggregate_egis_sunburst.html'
py.plot(fig, filename='assets/{0}'.format(filename), auto_open=False)
IFrame(src="./{0}".format(filename), width='100%', height=800)

In [27]:
# df = px.data.tips()

In [28]:
# fig = px.sunburst(df, path=['day', 'time', 'sex'], values='total_bill')
# fig.show()